In [1]:
import h5py
import numpy as np
import h5axeconfig
import polyclip

In [2]:
beamfile='/Users/kbhirombhakdi/_work/h5axeconfig-master/h5axeconfig/data/hst_wfc3_ir_beams.h5'
grism = 'G141'
conf = h5axeconfig.Camera(beamfile,grism)
for detname,detconf in conf:
    for beamname,beamconf in detconf:
        if beamname == 'A':
            break

In [3]:
lamb = np.array([9950., 10000.])
xd = np.array([487.,487.,488.,488.])
yd = np.array([546.,547.,547.,546.])
naxis = [1014,1014]
nx = 1014

# Reference

In [4]:
sref = beamconf.dispersion.arclength(lamb,xd,yd)
sref

array([[20.3772369 , 20.37560006, 20.37367402, 20.37531062],
       [21.44717203, 21.44547394, 21.4435543 , 21.44525215]])

In [5]:
xgref,ygref = beamconf.trace(sref,xd,yd)
print(xgref,ygref)

[[507.37634923 507.3747116  508.37278543 508.37442282]
 [508.44623774 508.44453883 509.44261905 509.44431773]] [[547.13208747 548.13030472 548.13037835 547.13216043]
 [547.14207417 548.14029567 548.14037053 547.14214834]]


In [6]:
xyg,lam,val = beamconf.specDrizzle(xd,yd,lamb)
if len(xyg)!=0:
    yl,xl=np.divmod(xyg,nx)
for i in np.arange(len(xyg)):
    print(lam[i],lamb[lam[i]],xl[i],yl[i],val[i])

0 9950.0 507 547 0.5418747
0 9950.0 507 548 0.08148474
0 9950.0 508 547 0.32432148
0 9950.0 508 548 0.048612647
1 10000.0 508 547 0.47568497
1 10000.0 508 548 0.07794053
1 10000.0 509 547 0.3805318
1 10000.0 509 548 0.06215349


# Breaking down: dispersion.arclength

This is to compute Eq. 4 from the LINEAR paper:

$$
\lambda(s) = \sum_{i=0}^n \beta_i(x_0,y_0)s^i
$$

These $\beta_i$ can be found as the dispersion coefficients in the .h5 files. Let's say we are dealing with G141. In the file, we will see two orders ($i=$0 and $i=$1 i.e., first-order polynomial). For each order, there are six orders ($j=$0 to $j=$5).

We compress six coefficients by following Eq. 7 from the LINEAR paper:

$$
\beta_i(x_0,y_0) = \sum_{j=0}^{n'} \sum_{k=0}^{n'} \beta_{i,j,k} x_0^j y_0^k \, ; \, j+k \leq n'
$$

Simply say, this is how you map the six coefficients for the case of G141, dispersion order $i=$0:

| j | k | Cjk |
|---|---|-----|
| 0 | 0 | 8951.386   |
| 1 | 0 | 0.080   |
| 1 | 1 | -0.009   |
| 2 | 0 | 2.185e-5   |
| 2 | 1 | -1.104e-5 |
| 2 | 2 | 3.352e-5 |

Then, the arclength $s$ will be the solution of the dispersion function:

$s(\lambda,x_0,y_0)$ = numpy.roots($\sum_{i=0}^n \beta_i(x_0,y_0)s^i - \lambda(s)$) .

In [7]:
h5 = h5py.File(beamfile,'r')
hf = h5['G141']['IR']['A']['dispersion']
print(hf.keys())

<KeysViewHDF5 ['0', '1']>


In [8]:
# Poly2d
def poly2d(v,xd,yd):
    coefs = {}
    if isinstance(v,np.ndarray):
        print('yes')
        print('len(v) = ',len(v))
        count = int((np.sqrt(1+8*len(v)) - 1)/2)
        print('count = ',count)
        ii = 0
        for j in range(count):
            print('j = ',j)
            for k in range(j+1):
                print('\tk = ',k)
                coefs[(j-k,k)] = v[ii]
                print('\t\tv = ',v[ii])
                ii+=1


    else:
        print('no')

    print('coefs = ',coefs,'\n')
    print('npar = len(coefs) = ',len(coefs),'\n')
    print('order = ',len(coefs)-1,'\n')
    
    p = 0
    for (i,j),v in coefs.items():
        p += v * xd**i * yd**j
    print('p = ',p)
    
    print('\n###################\n')
    
    
    return p

In [9]:
# Poly1d
coefs = []
for order in hf:
    print('order = ',order)
    x = hf[order][:]
    print(x)
    
    x = poly2d(x,xd,yd)
    
    coefs.append([x])
print(coefs)

order =  0
[ 8.95138621e+03  8.04403282e-02 -9.27969877e-03  2.18566417e-05
 -1.10480089e-05  3.35271254e-05]
yes
len(v) =  6
count =  3
j =  0
	k =  0
		v =  8951.38620572
j =  1
	k =  0
		v =  0.08044032819916265
	k =  1
		v =  -0.009279698766495334
j =  2
	k =  0
		v =  2.1856641668116504e-05
	k =  1
		v =  -1.1048008881387708e-05
	k =  2
		v =  3.352712538187608e-05
coefs =  {(0, 0): 8951.38620572, (1, 0): 0.08044032819916265, (0, 1): -0.009279698766495334, (2, 0): 2.1856641668116504e-05, (1, 1): -1.1048008881387708e-05, (0, 2): 3.352712538187608e-05} 

npar = len(coefs) =  6 

order =  5 

p =  [8997.73493273 8997.7569178  8997.85262509 8997.83065107]

###################

order =  1
[ 4.49722789e+01  4.92789151e-04  3.57824166e-03 -9.17523335e-07
  2.23550604e-07 -9.25869000e-07]
yes
len(v) =  6
count =  3
j =  0
	k =  0
		v =  44.97227893276267
j =  1
	k =  0
		v =  0.0004927891511929662
	k =  1
		v =  0.0035782416625653765
j =  2
	k =  0
		v =  -9.175233345083485e-07
	k =  1
		

In [10]:
# Dispersion
dldp = coefs
b = dldp[0]
m = dldp[1]
s = (lamb[:,None] - b)/m # because IR dispersion is a linear
s

array([[20.3772369 , 20.37560006, 20.37367402, 20.37531062],
       [21.44717203, 21.44547394, 21.4435543 , 21.44525215]])

In [11]:
s == sref

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])

### Verified!!! s = self.dispersion.arclength(lamb,xd,yd)

# Breaking down: trace

This computes the spectral trace (i.e., corners of a polygon given a source's segmentation corners $(x_0,y_0)$s and wavelength) following Eq. 2 and 3:

$$
x = x_0 + x_{off}(x_0,y_0) + \tilde{x} \\
y = y_0 + y_{off}(x_0,y_0) + \tilde{y}
$$

The $x_{off}, y_{off}$ are approximately constant -- zeroth order polynomial -- and can be found in the .h5 file. Therefore, we only need to compute $\tilde{x}, \tilde{y}$. This needs the arclength solutions $s$ from the previous step, and the trace coefficients in the .h5 file.

For the trace coefficients $\alpha_i$, we follow Eq. 1 and 5 in the LINEAR paper:

$$
s(\tilde{x}) = \int_0^{\tilde{x}} \sqrt{1 + \frac{d\tilde{y}}{dx'}} \, dx' \, .
$$

where

$$
\tilde{y}(\tilde{x}) = \sum_{i=0}^n \alpha_i(x_0,y_0) \tilde{x}^i
$$

and

$$
\alpha_i(x_0,y_0) = \sum_{j=0}^{n'} \sum_{k=0}^{n'} \alpha_{i,j,k} x_0^j y_0^k \, ; \, j+k \leq n'
$$

Similar to what we did with the dispersion, for G141 the trace is a first-order polynomial ($\alpha_0$ and $\alpha_1$). For each $\alpha_i$, it has six coefficients which can be compressed in a similar fashion as what we did with the dispersion. For G141/A/trace/0, we can arrange the six coefficients as:

| j | k | Cjk |
|---|---|-----|
| 0 | 0 | 2.081   |
| 1 | 0 | -1.975e-4   |
| 1 | 1 | -0.002   |
| 2 | 0 | 3.143e-8   |
| 2 | 1 | 4.321e-7 |
| 2 | 2 | 1.210e-7 |

Note that since G141's trace is a first-order polynomial, there applies its analytical solution from the integration.

In [12]:
def trace(s,coef):
    b = coef[0][0]
    m = coef[1][0]
    x = s/np.sqrt(1. + m*m)
    y = m*x + b
    return x,y

In [13]:
h5 = h5py.File(beamfile,'r')
hf = h5['G141']['IR']['A']['trace']
print(hf.keys())

dydx = []
for order in hf:
    print('order = ',order)
    x = hf[order][:]
    print(x)
    
    x = poly2d(x,xd,yd)
    
    dydx.append([x])
print(dydx)

coefs = dydx # from __call__

<KeysViewHDF5 ['0', '1']>
order =  0
[ 2.08196481e+00 -1.97521306e-04 -2.20206657e-03  3.14351408e-08
  4.32127869e-07  1.21043600e-07]
yes
len(v) =  6
count =  3
j =  0
	k =  0
		v =  2.08196481352
j =  1
	k =  0
		v =  -0.00019752130624389416
	k =  1
		v =  -0.002202066565067532
j =  2
	k =  0
		v =  3.143514082596283e-08
	k =  1
		v =  4.3212786880932414e-07
	k =  2
		v =  1.210435999122636e-07
coefs =  {(0, 0): 2.08196481352, (1, 0): -0.00019752130624389416, (0, 1): -0.002202066565067532, (2, 0): 3.143514082596283e-08, (1, 1): 4.3212786880932414e-07, (0, 2): 1.210435999122636e-07} 

npar = len(coefs) =  6 

order =  5 

p =  [0.94188773 0.94002841 0.94009791 0.9419568 ]

###################

order =  1
[ 1.02052817e-02 -6.06056924e-06 -3.24856004e-06  4.23638663e-10
  1.23095685e-08  1.61230739e-09]
yes
len(v) =  6
count =  3
j =  0
	k =  0
		v =  0.010205281672977665
j =  1
	k =  0
		v =  -6.06056923866002e-06
	k =  1
		v =  -3.2485600412356953e-06
j =  2
	k =  0
		v =  4.23638663

In [14]:
xoff = 0.
yoff = 0.

In [15]:
xhat,yhat = trace(s,coefs)
xg = xhat + xd + xoff
yg = yhat + yd + yoff
print(xg == xgref)
print(yg == ygref)

[[ True  True  True  True]
 [ True  True  True  True]]
[[ True  True  True  True]
 [ True  True  True  True]]


### Verified!!! xg,yg = self.trace(s,xd,yd)

# Calculating effective areas

This computes an effective area in each pixel bin given the segmentation corners. We do not go into details how this is implemented. See documentation for the polyclip package (by Russell) for more information.

In [16]:
xg = np.clip(xg,0,naxis[0])
yg = np.clip(yg,0,naxis[1])
print(xg==xgref,yg==ygref)

[[ True  True  True  True]
 [ True  True  True  True]] [[ True  True  True  True]
 [ True  True  True  True]]


In [17]:
x,y,area,indices = beamconf.polyclip(xg,yg)
print(x==xl,y==yl,area==val)

[ True  True  True  True  True  True  True  True] [ True  True  True  True  True  True  True  True] [ True  True  True  True  True  True  True  True]


### Verified!!!